In [2]:
from transformers import GPT2Config, GPT2Tokenizer
from datasets import load_dataset
import numpy as np
import zarr
from tqdm import tqdm
from datasets import Dataset

# Load the GPT2 configuration
config = GPT2Config()

# Load the tokenizer from the local directory
tokenizer = GPT2Tokenizer(
    vocab_file="./tokenizer/vocab.json",
    merges_file="./tokenizer/merges.txt"
)

# Configure tokenizer settings
tokenizer.model_max_length = config.n_positions
tokenizer.eos_token = tokenizer.bos_token
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

# Load the dataset
dataset = load_dataset("carolina-c4ai/corpus-carolina", taxonomy="wik")['corpus']

# Define the padding and truncation function
def tokenize_function(batch):
    return tokenizer(
        batch['text'],
        padding='max_length',  # Pad to the maximum length of the model
        truncation=True,       # Truncate if the text is longer than the maximum length
        max_length=config.n_positions,
        return_tensors='np'    # Return NumPy arrays
    )

# Create a Zarr group to store the tokenized data
zarr_save_path = 'dataset.zarr'
zarr_group = zarr.open_group(zarr_save_path, mode='w')
zarr_input_ids = zarr_group.create_dataset('input_ids', shape=(0, config.n_positions), maxshape=(None, config.n_positions), chunks=(1000, config.n_positions), dtype='int')
zarr_attention_mask = zarr_group.create_dataset('attention_mask', shape=(0, config.n_positions), maxshape=(None, config.n_positions), chunks=(1000, config.n_positions), dtype='int')

# Tokenize and save the dataset incrementally
batch_size = 1000  # Adjust batch size as needed
for i in tqdm(range(0, len(dataset), batch_size)):
    batch = dataset[i:i+batch_size]
    tokenized_batch = tokenize_function(batch)

    input_ids = tokenized_batch['input_ids']
    attention_mask = tokenized_batch['attention_mask']
    
    zarr_input_ids.append(input_ids)
    zarr_attention_mask.append(attention_mask)

print(f"Tokenized dataset saved to {zarr_save_path}")


_dict = {"input_ids": zarr_group.input_ids, "attention_mask": zarr_group.attention_mask}

/home/nando/miniconda3/envs/torch/lib/python3.11/site-packages/datasets/load.py:1486: FutureWarning: The repository for carolina-c4ai/corpus-carolina contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/carolina-c4ai/corpus-carolina
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
/home/nando/miniconda3/envs/torch/lib/python3.11/site-packages/zarr/creation.py:295: UserWarning: ignoring keyword argument 'maxshape'
  warn("ignoring keyword argument %r" % k)
100%|██████████| 961/961 [25:23<00:00,  1.59s/it]

Tokenized dataset saved to dataset.zarr


In [4]:
import tempfile
import os

# Create a temporary directory for saving chunks
temp_dir = tempfile.mkdtemp()

# Save the tokenized data in smaller chunks
chunk_size = 10000  # Adjust chunk size as needed
chunk_paths = []

for start in tqdm(range(0, len(zarr_input_ids), chunk_size)):
    end = start + chunk_size
    input_ids_chunk = np.array(zarr_input_ids[start:end])
    attention_mask_chunk = np.array(zarr_attention_mask[start:end])
    
    dataset_dict = {"input_ids": input_ids_chunk, "attention_mask": attention_mask_chunk}
    chunk_dataset = Dataset.from_dict(dataset_dict)
    
    chunk_path = os.path.join(temp_dir, f'chunk_{start//chunk_size}')
    chunk_dataset.save_to_disk(chunk_path)
    chunk_paths.append(chunk_path)

  0%|          | 0/97 [00:00<?, ?it/s]

Saving the dataset (0/1 shards):   0%|          | 0/10000 [00:00<?, ? examples/s]

  1%|          | 1/97 [00:00<01:09,  1.39it/s]

Saving the dataset (0/1 shards):   0%|          | 0/10000 [00:00<?, ? examples/s]

  2%|▏         | 2/97 [00:01<01:02,  1.52it/s]

...

Saving the dataset (0/1 shards):   0%|          | 0/10000 [00:00<?, ? examples/s]

 99%|█████████▉| 96/97 [00:56<00:00,  1.71it/s]

Saving the dataset (0/1 shards):   0%|          | 0/139 [00:00<?, ? examples/s]

100%|██████████| 97/97 [00:56<00:00,  1.73it/s]


AttributeError: 'Dataset' object has no attribute 'concatenate'

In [6]:
import datasets
dss = [Dataset.load_from_disk(chunk_path) for chunk_path in chunk_paths]

final_dataset = datasets.concatenate_datasets(dss)

In [7]:
final_dataset.save_to_disk("tokenized_dataset")

Saving the dataset (0/10 shards):   0%|          | 0/960139 [00:00<?, ? examples/s]

In [9]:
ds = datasets.load_from_disk("tokenized_dataset")